# Cleaning data

After a long wait to download the data files, it's high time to start exploring the data!  


In [4]:
import pandas as pd

# read the json file
file_path = '../data/conso.json'
df = pd.read_json(file_path)
# it's long...

In [5]:
df.head()

In [6]:
# save to pickle
df.to_pickle('../data/conso.pkl')

In [7]:
# read it back, muck faster than json
# and keep will keep types
df = pd.read_pickle('../data/conso.pkl')

In [8]:
# Sooooo, what's in there?
df.shape

(2121408, 32)

In [9]:
# First step is to drop columns with all NaN
# This is how we do it
df.dropna(axis=1, how='all', inplace=True)

In [10]:
df.shape  # One down !

(2121408, 31)

In [11]:
# Let's use the auto cast feature of pandas, sometimes it's working neat
df = df.convert_dtypes()

In [12]:
# And sometimes, it's not
df.dtypes

code_insee_region               Int64
libelle_region         string[python]
nature                 string[python]
date                   datetime64[ns]
heure                  string[python]
date_heure             string[python]
consommation                    Int64
thermique                       Int64
nucleaire                       Int64
eolien                 string[python]
solaire                         Int64
hydraulique                     Int64
pompage                         Int64
bioenergies                     Int64
ech_physiques                   Int64
stockage_batterie               Int64
destockage_batterie             Int64
eolien_terrestre                Int64
eolien_offshore                 Int64
tco_thermique                 Float64
tch_thermique                 Float64
tco_nucleaire                 Float64
tch_nucleaire                 Float64
tco_eolien                    Float64
tch_eolien                    Float64
tco_solaire                   Float64
tch_solaire 

In [13]:
# Here's the reason why the colum eolien is not a number
# Pandas won't cast it to a number because of the 'ND' string
# We will replace it with NaN, not by using .astype() or .replace()
# but by forcing the cast with pd.to_numeric(errors='coerce')
# This will replace any non number with NaN
df['eolien'].head(20)

0       ND
1       ND
2       ND
3       ND
4       ND
5       ND
6       ND
7       ND
8       ND
9       ND
10      ND
11      ND
12    1109
13     508
14      16
15     182
16     106
17     367
18     216
19      12
Name: eolien, dtype: string

In [14]:
# force cast to int for eolien column
df['eolien'] = pd.to_numeric(df['eolien'], errors='coerce').astype('Int64')

In [15]:
df['eolien'].head(20)

0     <NA>
1     <NA>
2     <NA>
3     <NA>
4     <NA>
5     <NA>
6     <NA>
7     <NA>
8     <NA>
9     <NA>
10    <NA>
11    <NA>
12    1109
13     508
14      16
15     182
16     106
17     367
18     216
19      12
Name: eolien, dtype: Int64

In [16]:
# I think this is the same for the date_heure column
# Since it's a UTC date, pandas seems to say "do it yourself, I don't want to break anything"
# Let's do it ourselves !
# First, let's check the data
df['date_heure'].head()  # UTC +1

0    2013-01-01T00:00:00+01:00
1    2013-01-01T00:00:00+01:00
2    2013-01-01T00:00:00+01:00
3    2013-01-01T00:00:00+01:00
4    2013-01-01T00:00:00+01:00
Name: date_heure, dtype: string

In [17]:
# Let's convert it to UTC
pd.to_datetime(df['date_heure'].head(), utc=True)
# Uh uh... the date is converted to UTC
# So 1 hour is removed from the date
# And the data won't start at midnight anymore !


0   2012-12-31 23:00:00+00:00
1   2012-12-31 23:00:00+00:00
2   2012-12-31 23:00:00+00:00
3   2012-12-31 23:00:00+00:00
4   2012-12-31 23:00:00+00:00
Name: date_heure, dtype: datetime64[ns, UTC]

In [18]:
# What's that ?
df.loc[df['date_heure'].str.contains('+02', regex=False), 'date_heure']  # Summer time !

51312      2013-03-31T03:00:00+02:00
51313      2013-03-31T03:00:00+02:00
51314      2013-03-31T03:00:00+02:00
51315      2013-03-31T03:00:00+02:00
51316      2013-03-31T03:00:00+02:00
                     ...            
2067307    2022-10-30T01:30:00+02:00
2067308    2022-10-30T01:30:00+02:00
2067309    2022-10-30T01:30:00+02:00
2067310    2022-10-30T01:30:00+02:00
2067311    2022-10-30T01:30:00+02:00
Name: date_heure, Length: 1229760, dtype: string

In [19]:
# We'll convert the date_heure column to UTC, then set the timezone to 'Europe/Paris' and we're done !
new_date = pd.to_datetime(df['date_heure'], utc=True).dt.tz_convert('Europe/Paris')  # this is how we do it
new_date.head()

0   2013-01-01 00:00:00+01:00
1   2013-01-01 00:00:00+01:00
2   2013-01-01 00:00:00+01:00
3   2013-01-01 00:00:00+01:00
4   2013-01-01 00:00:00+01:00
Name: date_heure, dtype: datetime64[ns, Europe/Paris]

In [20]:
# We're good !
df['date_heure'] = new_date

In [21]:
df.dtypes  # Nice !

code_insee_region                             Int64
libelle_region                       string[python]
nature                               string[python]
date                                 datetime64[ns]
heure                                string[python]
date_heure             datetime64[ns, Europe/Paris]
consommation                                  Int64
thermique                                     Int64
nucleaire                                     Int64
eolien                                        Int64
solaire                                       Int64
hydraulique                                   Int64
pompage                                       Int64
bioenergies                                   Int64
ech_physiques                                 Int64
stockage_batterie                             Int64
destockage_batterie                           Int64
eolien_terrestre                              Int64
eolien_offshore                               Int64
tco_thermiqu

In [22]:
# Let's check the data
df.head(20)

,code_insee_region,libelle_region,nature,date,heure,date_heure,consommation,thermique,nucleaire,eolien,...,tco_nucleaire,tch_nucleaire,tco_eolien,tch_eolien,tco_solaire,tch_solaire,tco_hydraulique,tch_hydraulique,tco_bioenergies,tch_bioenergies
0,84,Auvergne-Rhône-Alpes,Données définitives,2013-01-01,00:00,2013-01-01 00:00:00+01:00,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,44,Grand Est,Données définitives,2013-01-01,00:00,2013-01-01 00:00:00+01:00,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,27,Bourgogne-Franche-Comté,Données définitives,2013-01-01,00:00,2013-01-01 00:00:00+01:00,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,76,Occitanie,Données définitives,2013-01-01,00:00,2013-01-01 00:00:00+01:00,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,28,Normandie,Données définitives,2013-01-01,00:00,2013-01-01 00:00:00+01:00,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,93,Provence-Alpes-Côte d'Azur,Données définitives,2013-01-01,00:00,2013-01-01 00:00:00+01:00,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,53,Bretagne,Données définitives,2013-01-01,00:00,2013-01-01 00:00:00+01:00,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,11,Île-de-France,Données définitives,2013-01-01,00:00,2013-01-01 00:00:00+01:00,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,24,Centre-Val de Loire,Données définitives,2013-01-01,00:00,2013-01-01 00:00:00+01:00,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,32,Hauts-de-France,Données définitives,2013-01-01,00:00,2013-01-01 00:00:00+01:00,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [23]:
# It's seems we have some NaN in the consommation column
# We will drop the records with NaN in ths column, 
df['consommation'].isna().sum()

np.int64(12)

In [24]:
# drop rows where the column 'consommation' is NaN
df.dropna(subset=['consommation'], inplace=True)

In [25]:
df.head()

,code_insee_region,libelle_region,nature,date,heure,date_heure,consommation,thermique,nucleaire,eolien,...,tco_nucleaire,tch_nucleaire,tco_eolien,tch_eolien,tco_solaire,tch_solaire,tco_hydraulique,tch_hydraulique,tco_bioenergies,tch_bioenergies
12,44,Grand Est,Données définitives,2013-01-01,00:30,2013-01-01 00:30:00+01:00,4943,319,9137,1109,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
13,24,Centre-Val de Loire,Données définitives,2013-01-01,00:30,2013-01-01 00:30:00+01:00,2476,90,9085,508,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
14,11,Île-de-France,Données définitives,2013-01-01,00:30,2013-01-01 00:30:00+01:00,9134,685,<NA>,16,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
15,52,Pays de la Loire,Données définitives,2013-01-01,00:30,2013-01-01 00:30:00+01:00,3595,127,<NA>,182,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
16,27,Bourgogne-Franche-Comté,Données définitives,2013-01-01,00:30,2013-01-01 00:30:00+01:00,2357,236,<NA>,106,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [26]:
# consommatin is in MW, same for production

In [30]:
# sort the dataframes by date_heure ascending
df.sort_values('date_heure', ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [31]:
df.set_index('date_heure', inplace=True)

df.to_pickle('../data/conso-processed.pkl')